# Hotel Booking Cancellations Data Cleaning

In [27]:
import numpy as np
import pandas as pd
import pycountry

In [28]:
#import data
hotel = pd.read_csv("hotel_bookings copy.csv")
hotel.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [29]:
#Inspect df
hotel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [30]:
#Replace instances whre country is CN with CAN
hotel.loc[hotel['country'] == 'CN', 'country'] = 'CAN'

In [31]:
#Replace 3 letter country ISO code with country name
import pycountry

def iso_to_country(iso_code):
    try:
        country = pycountry.countries.get(alpha_3=iso_code)
        return country.name
    except:
        #print(iso_code)
        return None

hotel['country_name'] = hotel['country'].apply(iso_to_country)

In [32]:
#Convert date column to date time object
hotel["reservation_status_date"] = pd.to_datetime(hotel["reservation_status_date"])

In [33]:
#View earliest date
hotel["reservation_status_date"].min()

Timestamp('2014-10-17 00:00:00')

In [34]:
#View latest date
hotel["reservation_status_date"].max()

Timestamp('2017-09-14 00:00:00')

In [35]:
#View unique reservation status
hotel["reservation_status"].unique()

array(['Check-Out', 'Canceled', 'No-Show'], dtype=object)

In [36]:
#Create new column for change of rooms
hotel_room_change = hotel.loc[hotel["reserved_room_type"] != hotel["assigned_room_type"]]

In [37]:
#Obtain cancellations per country
cancelations_per_country = hotel_room_change.groupby("country_name")["is_canceled"].sum()

In [38]:
#Obtain bookings per country
bookings_per_country = hotel_room_change.groupby("country_name")["is_canceled"].count()

In [39]:
#Obtain percentage of cancellations per country
percentage_cancellations = pd.DataFrame(cancelations_per_country / bookings_per_country, index=None).reset_index()

In [40]:
percentage_cancellations.head()

,country_name,is_canceled
0,Algeria,0.142857
1,Angola,0.060606
2,Argentina,0.037037
3,Aruba,0.000000
4,Australia,0.000000


In [41]:
#percentage cancellations to csv
percentage_cancellations.to_csv("percentage_cancellations.csv", index=False)

In [42]:
#Obtain waiting days only for cancelled reservations
waiting_days = pd.DataFrame(hotel.groupby("days_in_waiting_list")["is_canceled"].sum()).reset_index().sort_values("days_in_waiting_list")

In [43]:
waiting_days.head()

,days_in_waiting_list,is_canceled
0,0,41865
1,1,3
2,2,1
3,3,59
4,4,8


In [44]:
#Max waiting days
waiting_days["days_in_waiting_list"].max()

391

In [45]:
waiting_days.to_csv("waiting_days.csv", index=False)